In [1]:
from bs4 import BeautifulSoup
import requests


class ZipScrapper:
    def __init__(self):
        self._relevant = {"Persons Per Householde:", "Average House Value:", "Income Per Household:", "Median Age:",
                          "Population:", "White Population:", "Black Population:", "Female Population:", "Male Population:"}
        
    @staticmethod
    def stripcontent(element):
        return element.contents[0].strip()

    def parseZip(self, zipnumber):
        r = requests.get("http://zipcode.org/{}".format(zipnumber))
        soup = BeautifulSoup(r.text, 'html.parser')
        res = {}
        for text in [f for f in soup.find_all("div", class_='Zip_Code_HTML_Block_Label') if ZipScrapper.stripcontent(f) in self._relevant]:
            key = ZipScrapper.stripcontent(text)
            key = key.replace(" ","").replace(":","")
            value = ZipScrapper.stripcontent(text.find_next_siblings(class_='Zip_Code_HTML_Block_Text')[0])
            value = value.replace("$","").replace(",","")
            res[key] = float(value)
        return res
            
            
z = ZipScrapper()
z.parseZip(48201)

{'AverageHouseValue': 93500.0,
 'BlackPopulation': 9260.0,
 'FemalePopulation': 6258.0,
 'IncomePerHousehold': 12262.0,
 'MalePopulation': 6556.0,
 'MedianAge': 36.4,
 'PersonsPerHouseholde': 1.66,
 'Population': 12814.0,
 'WhitePopulation': 2752.0}

In [1]:
from Zipscrapper import *
import sqlite3

detroitzip =[48201 ,48202 ,48206 ,48207 ,48208 ,48209 ,48213 ,48214 ,48215 ,48217 ,48219 ,48222 ,48223 ,
             48224 ,48204 ,48205 ,48210 ,48211 ,48216 ,48221 ,48226 ,48227 ,48228 ,48231 ,48238 ,48243 ,
             48244 ,48255 ,48265 ,48266 ,48269 ,48272 ,48275 ,48277 ,48232 ,48233 ,48234 ,48235 ,48242 ,
             48260 ,48264 ,48267 ,48268 ,48278 ,48279 ,48288]

query = "insert into zipdata (zipcode, PersonsPerHouseholde, AverageHouseValue, IncomePerHousehold, MedianAge, Population, WhitePopulation, BlackPopulation, MalePopulation, FemalePopulation) values (:zipcode, :PersonsPerHouseholde, :AverageHouseValue, :IncomePerHousehold, :MedianAge, :Population, :WhitePopulation, :BlackPopulation, :MalePopulation, :FemalePopulation)"

    
z = ZipScrapper()
con = sqlite3.connect("./data/location.s3db")
cur = con.cursor()
    
for zi in detroitzip:
    info = z.parseZip(zi)
    info['zipcode'] = zi
    print(info)
    cur.execute(query, info)
    con.commit()
con.close()

{'IncomePerHousehold': 12262.0, 'FemalePopulation': 6258.0, 'WhitePopulation': 2752.0, 'MalePopulation': 6556.0, 'Population': 12814.0, 'PersonsPerHouseholde': 1.66, 'BlackPopulation': 9260.0, 'MedianAge': 36.4, 'AverageHouseValue': 93500.0, 'zipcode': 48201}
{'IncomePerHousehold': 20708.0, 'FemalePopulation': 8457.0, 'WhitePopulation': 2270.0, 'MalePopulation': 8146.0, 'Population': 16603.0, 'PersonsPerHouseholde': 2.04, 'BlackPopulation': 13955.0, 'MedianAge': 35.3, 'AverageHouseValue': 67700.0, 'zipcode': 48202}
{'IncomePerHousehold': 24438.0, 'FemalePopulation': 11572.0, 'WhitePopulation': 688.0, 'MalePopulation': 10382.0, 'Population': 21954.0, 'PersonsPerHouseholde': 2.43, 'BlackPopulation': 21264.0, 'MedianAge': 38.0, 'AverageHouseValue': 53700.0, 'zipcode': 48206}
{'IncomePerHousehold': 24496.0, 'FemalePopulation': 11098.0, 'WhitePopulation': 1715.0, 'MalePopulation': 9154.0, 'Population': 20252.0, 'PersonsPerHouseholde': 1.85, 'BlackPopulation': 18437.0, 'MedianAge': 43.4, 'Av

In [37]:
from Zipscrapper import *
import sqlite3

#queryget = "select zipcode from zipdata where Population!=0;"
queryget = "select distinct zipcode from NewData where zipcode> 0 and zipcode  not in (select zipcode from zipdata)"
query = "insert into zipdatacomplete (	zipcode,PersonsPerHousehold,MedianHouseValue,IncomePerHousehold,PercentageFamilyHouseholds,PercentageBelowPoverty,PercentageBelow50Poverty,PercentageRenters,MedianAge,Population2013,Population2010,Population2000,WhitePopulation,BlackPopulation,MalePopulation,FemalePopulation,HouseholdNumber,FemalePopulation,UnemployedPercentage) values (	:zipcode, :PersonsPerHousehold, :MedianHouseValue, :IncomePerHousehold, :PercentageFamilyHouseholds, :PercentageBelowPoverty, :PercentageBelow50Poverty, :PercentageRenters, :MedianAge, :Population2013, :Population2010, :Population2000, :WhitePopulation, :BlackPopulation, :MalePopulation, :FemalePopulation, :HouseholdNumber, :FemalePopulation, :UnemployedPercentage)"
    

con = sqlite3.connect("./data/location.s3db")
cur = con.cursor()
cur.execute(queryget)

zips = [z[0] for z in cur.fetchall()]
z = ZipScrapper2()

for zi in zips:
    info = z.parseZip(zi)
    info['zipcode'] = zi
    print(info)
    cur.execute(query, info)
    con.commit()
con.close()

{'MedianHouseValue': 43900.0, 'WhitePopulation': 1470.0, 'IncomePerHousehold': 22804.0, 'PercentageBelow50Poverty': 21.8, 'Population2000': 45260.0, 'HouseholdNumber': 17113.0, 'PercentageRenters': 56.0, 'MalePopulation': 13115.0, 'zipcode': '48203', 'UnemployedPercentage': 31.4, 'MedianAge': 39.5, 'PercentageBelowPoverty': 42.8, 'Population2013': 26861.0, 'FemalePopulation': 13746.0, 'PercentageFamilyHouseholds': 41.0, 'Population2010': 28409.0, 'PersonsPerHousehold': 2.5, 'BlackPopulation': 25847.0}
{'MedianHouseValue': 40300.0, 'WhitePopulation': 14321.0, 'IncomePerHousehold': 23694.0, 'PercentageBelow50Poverty': 25.3, 'Population2000': 46136.0, 'HouseholdNumber': 16126.0, 'PercentageRenters': 42.0, 'MalePopulation': 20451.0, 'zipcode': '48212', 'UnemployedPercentage': 25.3, 'MedianAge': 31.1, 'PercentageBelowPoverty': 48.7, 'Population2013': 39369.0, 'FemalePopulation': 18918.0, 'PercentageFamilyHouseholds': 54.5, 'Population2010': 39038.0, 'PersonsPerHousehold': 3.3, 'BlackPopulat

AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [35]:
from Zipscrapper import *
import sqlite3
import time
from geopy.geocoders import Nominatim
#from geopy.geocoders import ArcGIS
import geopy.exc as geoexception

class BuildingInfoBuilder:
    def __init__(self):
        self.ok = True
    
    def GetInfo(self, buildname):
        if not self.ok:
            print('service error')
            return None
        try:
            time.sleep(2)           
            geolocator = Nominatim(timeout=2)
            location1 = geolocator.geocode(buildname, timeout=3)
            return location1.address.replace(", United States of America","")
        except (geoexception.GeocoderQuotaExceeded):
            self.ok = False
            print('service error')
            return None
        except Exception as error:
            print(buildname, " => ", error)
            return None



def BuildAdress(name, number):
    return "{} {}, Detroit, Michigan".format(number, name)


def GetZipcode(name):
    try:
        return int(name[-5:])
    except:
        return None

In [36]:
queryget = "select Id, StreetName,StreetNumber from NewData where ZipCode is null order by Demolished;"
query = "update NewData set ZipCode={} where Id={}"

con = sqlite3.connect("./data/location.s3db")
cur = con.cursor()
cur.execute(queryget)
builder = BuildingInfoBuilder()

for Id, street, number in cur.fetchall()[:]:
    rawname = BuildAdress(street, number)
    name = builder.GetInfo(rawname)
    if name is not None:
        zipcode = GetZipcode(name)
        print(zipcode, street, number, name)
        if zipcode is not None:
            cur.execute(query.format(zipcode,Id))
            con.commit()
con.close()

14941 Ardmore Avenue, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
17135 Ashton Avenue/Road, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
6359 Dale Avenue, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
19100 Park Street, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
1490 Campbell Avenue, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
5800 Warren Avenue, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
1352 Jefferson Avenue, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
17191 Alcoy Avenue/Street, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
12012 Wayburn Avenue, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
2400 Kendall Avenue, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
18481 Ashton Avenue/Road, Detroit, Michigan  =>  'NoneType' object has no attribute 'address'
6649 P

KeyboardInterrupt: 

In [1]:
from Zipscrapper import *
import sqlite3

#queryget = "select zipcode from zipdata where Population!=0;"
queryget = "select distinct zipcode from NewData where zipcode> 0 and zipcode  not in (select zipcode from zipdatacomplete)"
query = "insert into zipdatacomplete (	zipcode,PersonsPerHousehold,MedianHouseValue,IncomePerHousehold,PercentageFamilyHouseholds,PercentageBelowPoverty,PercentageBelow50Poverty,PercentageRenters,MedianAge,Population2013,Population2010,Population2000,WhitePopulation,BlackPopulation,MalePopulation,FemalePopulation,HouseholdNumber,FemalePopulation,UnemployedPercentage) values (	:zipcode, :PersonsPerHousehold, :MedianHouseValue, :IncomePerHousehold, :PercentageFamilyHouseholds, :PercentageBelowPoverty, :PercentageBelow50Poverty, :PercentageRenters, :MedianAge, :Population2013, :Population2010, :Population2000, :WhitePopulation, :BlackPopulation, :MalePopulation, :FemalePopulation, :HouseholdNumber, :FemalePopulation, :UnemployedPercentage)"
    

con = sqlite3.connect("./data/location.s3db")
cur = con.cursor()
cur.execute(queryget)

zips = [z[0] for z in cur.fetchall()][::-1]
z = ZipScrapper2()

for zi in zips:
    print(zi)
    info = z.parseZip(zi)
    info['zipcode'] = zi
    print(info)
    cur.execute(query, info)
    con.commit()
con.close()

48033
Zip code population in 2000 Population2000
Zip code population in 2010: Population2010
{'MalePopulation': 7128.0, 'UnemployedPercentage': 15.9, 'Population2010': 0, 'IncomePerHousehold': 44011.0, 'HouseholdNumber': 8450.0, 'Population2013': 16553.0, 'PercentageBelow50Poverty': 9.4, 'PercentageBelowPoverty': 18.8, 'PersonsPerHousehold': 2.2, 'Population2000': 0, 'MedianAge': 42.7, 'MedianHouseValue': 99100.0, 'PercentageRenters': 61.0, 'PercentageFamilyHouseholds': 39.1, 'FemalePopulation': 9425.0}
{'MalePopulation': 7128.0, 'UnemployedPercentage': 15.9, 'Population2010': 0, 'IncomePerHousehold': 44011.0, 'HouseholdNumber': 8450.0, 'WhitePopulation': 0, 'zipcode': '48033', 'Population2013': 16553.0, 'PercentageBelow50Poverty': 9.4, 'PercentageBelowPoverty': 18.8, 'BlackPopulation': 0, 'PersonsPerHousehold': 2.2, 'Population2000': 0, 'MedianAge': 42.7, 'MedianHouseValue': 99100.0, 'PercentageRenters': 61.0, 'PercentageFamilyHouseholds': 39.1, 'FemalePopulation': 9425.0}
